In [ ]:
import os

os.chdir('/home/jinho93/ml/gallium-oxide/pristine/beta/mlff')

: 

In [2]:
from glob import glob

from pymatgen.io.vasp.outputs import Vasprun


from monty.serialization import loadfn

data = loadfn('data.json')
train_structures = [d['structure'] for d in data]
train_energies = [d['outputs']['energy'] for d in data]
train_forces = [d['outputs']['forces'] for d in data]

def get_document(n, vrun):
    structure = vrun.ionic_steps[n]['structure']
    energy = vrun.ionic_steps[n]['e_wo_entrp']
    forces = vrun.ionic_steps[n]['forces']
    return structure, energy, forces

train_structures = []
train_energies = []
train_forces = []

vrun_files = glob("vasprun*")
vrun = Vasprun(vrun_files[0])
for v in range(vrun.nionic_steps):
    s, e, f = get_document(v, vrun)
    train_structures.append(s)
    train_energies.append(e)
    train_forces.append(f)

In [4]:
import numpy as np
from maml.utils import pool_from, convert_docs

train_pool = pool_from(train_structures, train_energies, train_forces)
_, df = convert_docs(train_pool)

weights = np.ones(len(df['dtype']), )

# set the weights for energy equal to 100
weights[df['dtype'] == 'energy'] = 10

In [5]:
from maml.base import SKLModel
from maml.describers import BispectrumCoefficients
from sklearn.linear_model import LinearRegression
from maml.apps.pes import SNAPotential

element_profile = {'Ga': {'r': 5.0, 'w': 1}, 'O': {'r': 5.0, 'w': 1}}
describer = BispectrumCoefficients(rcutfac=0.5, twojmax=6, element_profile=element_profile, 
                                   quadratic=False, pot_fit=True)
model = SKLModel(describer=describer, model=LinearRegression())
snap = SNAPotential(model=model)

# snap.set_lmp_exe('/home/jinho93/Downloads/n2p2/bin/lmp_mpi')

snap.train(train_structures, train_energies, train_forces, sample_weight=weights)

snap.write_param()

/home/jinho93/micromamba/envs/mlph/lib/python3.10/site-packages/maml/apps/pes/_lammps.py:35: FutureWarning: which is deprecated; use which in shutil instead.
shutil.which has been available since Python 3.3. This will be removed in v2023.
  if which(lmp_exe) is not None:
INFO:maml.apps.pes._lammps:Setting Lammps executable to lmp_serial
/home/jinho93/micromamba/envs/mlph/lib/python3.10/site-packages/maml/apps/pes/_lammps.py:82: FutureWarning: which is deprecated; use which in shutil instead.
shutil.which has been available since Python 3.3. This will be removed in v2023.
  if not which(lmp_exe):
INFO:maml.utils._lammps:Structure index 0 is rotated.
INFO:maml.utils._lammps:Structure index 1 is rotated.
INFO:maml.utils._lammps:Structure index 2 is rotated.
INFO:maml.utils._lammps:Structure index 3 is rotated.
INFO:maml.utils._lammps:Structure index 4 is rotated.
INFO:maml.utils._lammps:Structure index 5 is rotated.
INFO:maml.utils._lammps:Structure index 6 is rotated.
INFO:maml.utils._la

In [3]:
from maml.apps.pes import SNAPotential

snap = SNAPotential.from_config('SNAPotential.snapparam', 'SNAPotential.snapcoeff')

/home/jinho93/micromamba/envs/mlph/lib/python3.10/site-packages/maml/apps/pes/_lammps.py:35: FutureWarning: which is deprecated; use which in shutil instead.
shutil.which has been available since Python 3.3. This will be removed in v2023.
  if which(lmp_exe) is not None:
INFO:maml.apps.pes._lammps:Setting Lammps executable to lmp_serial
/home/jinho93/micromamba/envs/mlph/lib/python3.10/site-packages/maml/apps/pes/_lammps.py:82: FutureWarning: which is deprecated; use which in shutil instead.
shutil.which has been available since Python 3.3. This will be removed in v2023.
  if not which(lmp_exe):


In [5]:

from phonopy import Phonopy
import numpy as np

from phonopy.interface.calculator import read_crystal_structure
unitcell, _ = read_crystal_structure("POSCAR", interface_mode='vasp')

phonon = Phonopy(unitcell,
                 np.identity(3) * 4,
#                  primitive_matrix=np.reshape('0.0 0.5 0.5  0.5 0.0 0.5  0.5 0.5 0.0'.split(), (3, 3)))
                  primitive_matrix=np.identity(3))